# (5) Demo model 
* team20 (안지은, 이정윤)

In [19]:
import pickle
import json
import sys
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

# from flask import Flask, render_template, request
# from flask_restx import Api, Resource
# from flask_cors import CORS
# import sys
# import os

In [13]:
with open('./model/kmeans_500_model_final.pkl', 'rb') as file1:
    kmeans_model = pickle.load(file1)
    
with open('./model/knn_2_model_final.pkl', 'rb') as file2:
    knn_model = pickle.load(file2)
    
with open('./model/ols_model_final.pkl', 'rb') as file3:
   ols_model = pickle.load(file3)

with open('./data/result_bygroup_final.json', 'r') as file4:
    json_data = json.load(file4)
    
df = pd.read_csv('./data/df_formodel_final.csv', index_col=0)

/Users/jeongyunl/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator KMeans from version 1.3.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 434985 entries, 0 to 434984
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          434985 non-null  int64  
 1   condition     434985 non-null  float64
 2   odometer      434985 non-null  float64
 3   mmr           434985 non-null  int64  
 4   sellingprice  434985 non-null  int64  
dtypes: float64(2), int64(3)
memory usage: 19.9 MB


In [16]:
## K-means 모델을 이용한 그룹 분류

def make_ml_answer(*args,**kwargs):

    new_data = {'year': 2015, 'condition': 2, 'odometer': 100000, 'mmr': 10000, 'sellingprice': 10000}

    # DataFrame의 마지막 행에 값을 추가
    df.loc[len(df)] = new_data

    minmax_scaler = MinMaxScaler() # MinMaxScaler 
    
    minmax_scaler.fit(df) # 정규화를 수행 함수 생성
    transformed = minmax_scaler.transform(df) 
    
    df_scaled= pd.DataFrame(transformed)
    df_scaled.columns = df.columns
    
    predict_group = str(int(kmeans_model.predict(pd.DataFrame(df_scaled.loc[len(df_scaled)-1]).T.reset_index(drop=True))))
    predict_price = ols_model.predict(df.loc[(len(df)-1):,['year', 'condition', 'mmr', 'odometer']].reset_index(drop=True))

    result_predict_price = str(round(predict_price[0],2))
    
    result_year = json_data[predict_group]['count_year']
    result_condition = json_data[predict_group]['mean_condition']
    result_make = json_data[predict_group]['count_make']
    result_odometer = json_data[predict_group]['mean_odometer']
    result_mmr = json_data[predict_group]['mean_mmr']
    result_sellingprice = json_data[predict_group]['mean_sellingprice']
    
    output = {
        'year' : 2023 - result_year,
        'make' : result_make,
        'condition' : round(result_condition, 2),
        'odometer' : round(result_odometer, 2) ,
        'mmr' : round(result_mmr, 2),
        'sellingprice' :round(result_sellingprice, 2),
        'predict_price' : result_predict_price
    }                        

    return output

make_ml_answer()


/Users/jeongyunl/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


{'year': 11,
 'make': 'Nissan',
 'condition': 1.98,
 'odometer': 56985.56,
 'mmr': 13908.89,
 'sellingprice': 12279.02,
 'predict_price': '9452.18'}

In [21]:
## KNN 모델을 이용한 그룹 분류

def pca(df):
    pca = PCA(n_components=1)
    principalComponents = pca.fit_transform(df)
    principalDf = pd.DataFrame(data=principalComponents, columns = ['pc1'])
    return principalDf
    
def make_ml_answer(*args,**kwargs):

        new_data = {'year': 2010, 'condition': 2, 'odometer': 100000, 'mmr': 10000, 'sellingprice': 10000}

        # DataFrame의 마지막 행에 값을 추가
        df.loc[len(df)] = new_data
        
        minmax_scaler = MinMaxScaler() # MinMaxScaler 
        
        minmax_scaler.fit(df) # 정규화를 수행 함수 생성
        transformed = minmax_scaler.transform(df) 
        
        df_scaled= pd.DataFrame(transformed)
        df_scaled.columns = df.columns
        
        df_pca = pca(pd.DataFrame(df_scaled.loc[len(df_scaled)-1]).T.reset_index(drop=True))
        
        x_train_final = np.array(df_pca['pc1']).reshape(-1, 1)
        
        predict_group = str(int(knn_model.predict(x_train_final)))
        
        predict_price = ols_model.predict(df.loc[(len(df)-1):,['year', 'condition', 'mmr', 'odometer']].reset_index(drop=True))

        result_predict_price = str(round(predict_price[0],2))
        
        result_year = json_data[predict_group]['count_year']
        result_condition = json_data[predict_group]['mean_condition']
        result_make = json_data[predict_group]['count_make']
        result_odometer = json_data[predict_group]['mean_odometer']
        result_mmr = json_data[predict_group]['mean_mmr']
        result_sellingprice = json_data[predict_group]['mean_sellingprice']
        
        output = {
            'year' : 2023 - result_year,
            'make' : result_make,
            'condition' : round(result_condition, 2),
            'odometer' : round(result_odometer, 2) ,
            'mmr' : round(result_mmr, 2),
            'sellingprice' :round(result_sellingprice, 2),
            'predict_price' : result_predict_price
        }                        
    
        return output
    
make_ml_answer()


/Users/jeongyunl/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_pca.py:499: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)
/Users/jeongyunl/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


{'year': 16,
 'make': 'BMW',
 'condition': 4.23,
 'odometer': 73586.54,
 'mmr': 10965.11,
 'sellingprice': 12236.44,
 'predict_price': '9452.44'}

In [4]:
import pickle
import json
import sys
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
# from flask import Flask, render_template, request
# from flask_restx import Api, Resource
# from flask_cors import CORS
# import sys
# import os
# sys.path.append(os.getcwd())

########## data load ##########
with open('./model/kmeans_500_model_final.pkl', 'rb') as file1:
    kmeans_model = pickle.load(file1)
    
with open('./model/knn_2_model_final.pkl', 'rb') as file2:
    knn_model = pickle.load(file2)
    
with open('./model/ols_model_final.pkl', 'rb') as file3:
   ols_model = pickle.load(file3)

with open('./data/result_bygroup_final.json', 'r') as file4:
    json_data = json.load(file4)
    
df = pd.read_csv('./data/df_formodel_final.csv', index_col=0)

########## data preprocessing ##########
def pca(df):
    pca = PCA(n_components=1)
    principalComponents = pca.fit_transform(df)
    principalDf = pd.DataFrame(data=principalComponents, columns = ['pc1'])
    return principalDf

# app = Flask(__name__)
# CORS(app, resources={r"/*": {"origins": "http://127.0.0.1:8081"}})
# api = Api(app)

# @app.route('/', methods=['GET', 'POST']) 

def make_ml_answer(*args,**kwargs):
    # if request.method == 'POST':
        # HTML에서 POST 요청으로 받은 데이터 처리
        # year = request.form.get('year')
        # condition = request.form.get('condition')
        # odometer = request.form.get('odometer')
        # mmr = request.form.get('mmr')
        # sellingprice = request.form.get('sellingprice')
        
        new_data = {'year': 2008, 'condition': 3, 'odometer': 100000, 'mmr': 11000, 'sellingprice': 10000}
        
        # 정규화 진행
        df.loc[len(df)] = new_data
        minmax_scaler = MinMaxScaler() 
        minmax_scaler.fit(df) 
        transformed = minmax_scaler.transform(df) 
        df_scaled= pd.DataFrame(transformed)
        df_scaled.columns = df.columns
        
        # pca 진행
        df_pca = pca(pd.DataFrame(df_scaled.loc[len(df_scaled)-1]).T.reset_index(drop=True))
        x_train_final = np.array(df_pca['pc1']).reshape(-1, 1)
        
        # knn 모델 예측
        predict_group = str(int(knn_model.predict(x_train_final)))
        # ols 모델 예측
        predict_price = ols_model.predict(df.loc[(len(df)-1):,['year', 'condition', 'mmr', 'odometer']].reset_index(drop=True))
        result_predict_price = str(round(predict_price[0],2))
        
        # json 데이터에서 예측된 그룹의 정보 추출
        result_year = json_data[predict_group]['count_year']
        result_condition = json_data[predict_group]['mean_condition']
        result_make = json_data[predict_group]['count_make']
        result_odometer = json_data[predict_group]['mean_odometer']
        result_mmr = json_data[predict_group]['mean_mmr']
        result_sellingprice = json_data[predict_group]['mean_sellingprice']
        
        # 결과값 출력
        output = {
            'year' : 2023 - result_year,
            'make' : result_make,
            'condition' : round(result_condition, 2),
            'odometer' : round(result_odometer, 2) ,
            'mmr' : round(result_mmr, 2),
            'sellingprice' :round(result_sellingprice, 2),
            'predict_price' : result_predict_price
        }                        
    
        return  output

make_ml_answer()

/Users/jeongyunl/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator KMeans from version 1.3.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/jeongyunl/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_pca.py:499: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)
/Users/jeongyunl/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be el

{'year': 16,
 'make': 'BMW',
 'condition': 4.23,
 'odometer': 73586.54,
 'mmr': 10965.11,
 'sellingprice': 12236.44,
 'predict_price': '10397.62'}

In [7]:
predict_price = ols_model.predict(df.loc[(len(df)-1):,['year', 'condition', 'mmr', 'odometer']].reset_index(drop=True))
result_predict_price = str(round(predict_price[0],2))
predict_price

0    10397.616959
dtype: float64

In [3]:
import pandas as pd
df = pd.read_csv('./data/df_formodel_final.csv', index_col=0)
df = df[['year', 'condition', 'sellingprice', 'odometer', 'mmr']]
df

,year,condition,sellingprice,odometer,mmr
0,2015,5.0,21500,16639.0,20500
1,2015,5.0,21500,9393.0,20800
2,2014,4.5,30000,1331.0,31900
3,2015,4.1,27750,14282.0,27500
4,2015,1.0,10900,5554.0,15350
...,...,...,...,...,...
434980,2011,3.9,22800,66403.0,20300
434981,2012,5.0,30800,54393.0,30200
434982,2012,4.8,34000,50561.0,29800
434983,2015,3.8,11100,16658.0,15100
